#  Web Search Agent

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_wuzzuf(query="AI", location="Egypt", max_pages=3):
    jobs = []
    base_url = f"https://wuzzuf.net/search/jobs/?q={query}&a=hpb&l={location}"

    for page in range(max_pages):
        start = page * 10
        url = base_url + f"&start={start}"
        print(f"Scraping page {page+1}: {url}")
        resp = requests.get(url)
        if resp.status_code != 200:
            print(f"Failed to retrieve page {page+1}")
            break

        soup = BeautifulSoup(resp.text, "html.parser")
        job_cards = soup.find_all('div', class_='css-1gatmva e1v1l3u10') 

        for card in job_cards:
            try:
                title = card.find('h2').text.strip()
                company = card.find('a', class_='css-17s97q8').text.strip()
                loc = card.find('span', class_='css-5wys0k').text.strip()
                link = card.find('a')['href']

               
                if link.startswith('http'):
                    full_link = link
                else:
                    full_link = "https://wuzzuf.net" + link

                jobs.append({
                    'title': title,
                    'company': company,
                    'location': loc,
                    'link': full_link
                })
            except Exception as e:
                print(f"Error parsing job card: {e}")
                continue

        time.sleep(1)  

    return jobs

if __name__ == "__main__":
    data = scrape_wuzzuf(query="Machine Learning", location="Egypt", max_pages=2)
    print(f"Total jobs scraped: {len(data)}")
    for job in data[:3]:  # عرض أول 3 وظائف كمثال
        print(job)


Scraping page 1: https://wuzzuf.net/search/jobs/?q=Machine Learning&a=hpb&l=Egypt&start=0
Error parsing job card: 'href'
Scraping page 2: https://wuzzuf.net/search/jobs/?q=Machine Learning&a=hpb&l=Egypt&start=10
Total jobs scraped: 14
{'title': 'Data Science Instructor', 'company': 'EpsilonAI -', 'location': 'Nasr City, Cairo, Egypt', 'link': 'https://wuzzuf.net/jobs/careers/EpsilonAI-Egypt-65111'}
{'title': 'Senior AI Engineer', 'company': 'Lumin -', 'location': 'Sheikh Zayed, Giza, Egypt', 'link': 'https://wuzzuf.net/jobs/careers/Excel-Systems-LLC-Egypt-13729'}
{'title': 'Research Engineer', 'company': 'Enoshsciense Research Center -', 'location': 'New Cairo, Cairo, Egypt', 'link': 'https://wuzzuf.net/jobs/careers/Enoshsciense Research Center-Egypt-127306'}


# Data Extraction Agent

In [ ]:
def get_job_description(job_url):
    try:
        resp = requests.get(job_url)
        if resp.status_code != 200:
            print(f"Failed to fetch job description: {job_url}")
            return ""

        soup = BeautifulSoup(resp.text, "html.parser")

       
        desc_div = soup.find('div', class_='css-1m4cuuf') 

        if desc_div:
            description = desc_div.get_text(separator=' ').strip()
            return description
        else:
            return ""

    except Exception as e:
        print(f"Error fetching job description: {e}")
        return ""


In [12]:
def scrape_wuzzuf_with_desc(query="AI", location="Egypt", max_pages=2):
    jobs = []
    base_url = f"https://wuzzuf.net/search/jobs/?q={query}&a=hpb&l={location}"

    for page in range(max_pages):
        start = page * 10
        url = base_url + f"&start={start}"
        print(f"Scraping page {page+1}: {url}")
        resp = requests.get(url)
        if resp.status_code != 200:
            print(f"Failed to retrieve page {page+1}")
            break

        soup = BeautifulSoup(resp.text, "html.parser")
        job_cards = soup.find_all('div', class_='css-1gatmva e1v1l3u10')

        for card in job_cards:
            try:
                title = card.find('h2').text.strip()
                company = card.find('a', class_='css-17s97q8').text.strip()
                loc = card.find('span', class_='css-5wys0k').text.strip()
                link = card.find('a')['href']

                if link.startswith('http'):
                    full_link = link
                else:
                    full_link = "https://wuzzuf.net" + link

                description = get_job_description(full_link)

                jobs.append({
                    'title': title,
                    'company': company,
                    'location': loc,
                    'link': full_link,
                    'description': description
                })
            except Exception as e:
                print(f"Error parsing job card: {e}")
                continue

        time.sleep(1)
    return jobs

if __name__ == "__main__":
    jobs_data = scrape_wuzzuf_with_desc(query="Machine Learning", location="Egypt", max_pages=1)
    for job in jobs_data[:3]:
        print(job['title'])
        print(job['description'][:300])  # عرض أول 300 حرف من الوصف
        print('-'*50)


Scraping page 1: https://wuzzuf.net/search/jobs/?q=Machine Learning&a=hpb&l=Egypt&start=0
Error parsing job card: 'href'
Data Science Instructor

--------------------------------------------------
Senior AI Engineer

--------------------------------------------------
Research Engineer

--------------------------------------------------


# Trend Analysis Agent

In [ ]:
def extract_skills_from_description(description, skill_keywords):
    found_skills = []
    desc_lower = description.lower()
    for skill in skill_keywords:
        if skill.lower() in desc_lower:
            found_skills.append(skill)
    return list(set(found_skills)) 


In [ ]:
from collections import Counter

def analyze_jobs(jobs, skill_keywords):
    titles = [job['title'] for job in jobs]
    locations = [job['location'] for job in jobs]

   
    all_skills = []
    for job in jobs:
        skills = extract_skills_from_description(job.get('description', ''), skill_keywords)
        all_skills.extend(skills)


    title_counts = Counter(titles)
    skill_counts = Counter(all_skills)
    location_counts = Counter(locations)

    return title_counts.most_common(), skill_counts.most_common(), location_counts.most_common()


In [15]:
if __name__ == "__main__":
  
    jobs_data = scrape_wuzzuf_with_desc(query="Machine Learning", location="Egypt", max_pages=2)

    skill_keywords = ["Python", "TensorFlow", "PyTorch", "Deep Learning", "NLP", "SQL", "Docker", "Keras", "Scikit-learn"]

    top_titles, top_skills, top_locations = analyze_jobs(jobs_data, skill_keywords)

    print("Top Job Titles:")
    for title, count in top_titles[:10]:
        print(f"{title}: {count}")

    print("\nTop Skills:")
    for skill, count in top_skills[:10]:
        print(f"{skill}: {count}")

    print("\nJob Locations Distribution:")
    for loc, count in top_locations:
        print(f"{loc}: {count}")


Scraping page 1: https://wuzzuf.net/search/jobs/?q=Machine Learning&a=hpb&l=Egypt&start=0
Error parsing job card: 'href'
Scraping page 2: https://wuzzuf.net/search/jobs/?q=Machine Learning&a=hpb&l=Egypt&start=10
Top Job Titles:
Data Science Instructor: 1
Senior AI Engineer: 1
Research Engineer: 1
Senior AI Vision Engineer – Computer Vision & Edge Deployment: 1
Artificial Intelligence Engineer (NLP): 1
AI Trainer.: 1
Cloud Infrastructure Engineer: 1
Fresh Electrical PLC & Planning Maintenace Engineer: 1
AI Engineer: 1
AI Solutions Architect: 1

Top Skills:

Job Locations Distribution:
New Cairo, Cairo, Egypt: 2
Heliopolis, Cairo, Egypt: 2
Nasr City, Cairo, Egypt: 1
Sheikh Zayed, Giza, Egypt: 1
6th of October, Giza, Egypt: 1
Ain Sokhna, Suez, Egypt: 1
Maadi, Cairo, Egypt: 1
Cairo, Egypt: 1
Giza, Egypt: 1
Makkah, Saudi Arabia: 1
Dubai, United Arab Emirates: 1
Riyadh, Saudi Arabia: 1


#  Report Writer Agent

In [16]:
def generate_markdown_report(top_roles, top_skills, location_distribution, trends_summary):
    md = "# Top AI/ML Jobs in MENA – May 2025\n\n"

    md += "## Top 10 AI/ML Roles\n"
    for i, (role, count) in enumerate(top_roles[:10], 1):
        md += f"{i}. {role} ({count} postings)\n"

    md += "\n## Key Skills Required\n"
    md += ", ".join([skill for skill, _ in top_skills[:15]]) + "\n"

    md += "\n## Country-wise Job Distribution\n"
    for country, count in location_distribution:
        md += f"- {country}: {count} jobs\n"

    md += "\n## Trends & Observations\n"
    md += trends_summary + "\n"

    return md


In [17]:
def create_trends_summary(top_roles, top_skills):
    summary = f"The role '{top_roles[0][0]}' is currently the most demanded position, with {top_roles[0][1]} job postings. "
    summary += f"Key skills in demand include {', '.join([skill for skill, _ in top_skills[:5]])}. "
    summary += "The market shows strong growth in AI/ML jobs across the MENA region, especially in Egypt, UAE, and Saudi Arabia."
    return summary


In [ ]:
if __name__ == "__main__":
   
    top_roles = [('Machine Learning Engineer', 120), ('Data Scientist', 80), ('AI Researcher', 40)]
    top_skills = [('Python', 90), ('TensorFlow', 75), ('Deep Learning', 60), ('NLP', 55), ('SQL', 50)]
    location_distribution = [('Egypt', 150), ('UAE', 70), ('Saudi Arabia', 60)]

    trends_summary = create_trends_summary(top_roles, top_skills)
    report_md = generate_markdown_report(top_roles, top_skills, location_distribution, trends_summary)

    print(report_md)


# Top AI/ML Jobs in MENA – May 2025

## Top 10 AI/ML Roles
1. Machine Learning Engineer (120 postings)
2. Data Scientist (80 postings)
3. AI Researcher (40 postings)

## Key Skills Required
Python, TensorFlow, Deep Learning, NLP, SQL

## Country-wise Job Distribution
- Egypt: 150 jobs
- UAE: 70 jobs
- Saudi Arabia: 60 jobs

## Trends & Observations
The role 'Machine Learning Engineer' is currently the most demanded position, with 120 job postings. Key skills in demand include Python, TensorFlow, Deep Learning, NLP, SQL. The market shows strong growth in AI/ML jobs across the MENA region, especially in Egypt, UAE, and Saudi Arabia.



# CLI Preview

In [19]:
def preview_report(report_md, max_lines=20):
    lines = report_md.split('\n')
    preview_lines = lines[:max_lines]
    print("\n".join(preview_lines))
    if len(lines) > max_lines:
        print("\n... (Report truncated for preview) ...\n")


In [ ]:
if __name__ == "__main__":
   
    preview_report(report_md, max_lines=30)


# Top AI/ML Jobs in MENA – May 2025

## Top 10 AI/ML Roles
1. Machine Learning Engineer (120 postings)
2. Data Scientist (80 postings)
3. AI Researcher (40 postings)

## Key Skills Required
Python, TensorFlow, Deep Learning, NLP, SQL

## Country-wise Job Distribution
- Egypt: 150 jobs
- UAE: 70 jobs
- Saudi Arabia: 60 jobs

## Trends & Observations
The role 'Machine Learning Engineer' is currently the most demanded position, with 120 job postings. Key skills in demand include Python, TensorFlow, Deep Learning, NLP, SQL. The market shows strong growth in AI/ML jobs across the MENA region, especially in Egypt, UAE, and Saudi Arabia.

